In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import nltk
nltk.download("book")
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from bs4 import BeautifulSoup

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/mizanul/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package depend

In [2]:
df = pd.read_csv("./ntbmall-produk-desc.csv")
df = pd.DataFrame(df)
df

,ID Produk,Tanggal Input,Nama Toko,Email Pelapak,Kategori Produk,Sub Kategori Produk,Nama Produk,Variasi,Harga,Stok,Deksripsi,Unnamed: 11,Unnamed: 12
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,2024-04-13 04:24:29,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Permen,Supermen,40 Gr,12000,0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60 Gr,16000,0,<p>Permen susu khas lombok nusa tenggara barat...,NaN,NaN
2,b70def04-da59-4c2e-912a-e5944579831d,2023-10-29 07:45:51,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Coklat,Semeton Choco (Coklat Lombok),NaN,15000,37,NaN,<p>Coklat susu adalah mahakarya gurih yang mem...,NaN
3,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,2023-10-19 09:45:43,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Lanang,NaN,19000,20,<p>Kopi Lanang Bubuk Ground&nbsp;<br>Level Gri...,NaN,NaN
4,9271605a-009d-4364-8deb-f8d04847e38b,2023-10-19 09:35:33,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Lombok Murni,NaN,28000,20,<p>Kopi denga kemasaran Pouch <br>100% Tanpa C...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1911,bbfe46af-ca0a-4335-a329-67925c72d4fd,2021-03-25 13:42:44,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Madu,Mbojo Honey,NaN,169000,0,<p>Mbojo Honey<br></p>,NaN,NaN
1912,f2d75215-debe-4e1c-bcea-a3aaa468b50c,2021-03-25 13:38:11,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Bubuk 100 Gr,NaN,29000,0,<p>Kopi Bubuk Pesanggerahan 100 Gr</p><p><span...,NaN,NaN
1913,cb760741-76be-4c4b-99ee-0b4c671f0c8a,2021-03-25 13:34:01,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Jahe 100 Gr,NaN,29000,0,<p>Kopi Jahe Pesanggerahan 100 Gr</p><p><span ...,NaN,NaN
1914,3b90f8a5-bbef-466f-94a8-ce275b3fee70,2021-03-25 13:32:03,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Fashion & Bag,Fashion Wanita,Dompet Motif Krealang Sumbawa,NaN,455000,5,<p>Dompet Motif Krealang Sumbawa<br></p>,NaN,NaN


In [3]:
df_preprocess  = df.drop(["ID Produk", "Tanggal Input", "Nama Toko", "Email Pelapak", "Kategori Produk", "Variasi", "Stok", "Unnamed: 11", "Unnamed: 12"], axis=1)
df_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Sub Kategori Produk  1810 non-null   object
 1   Nama Produk          1810 non-null   object
 2   Harga                1916 non-null   int64 
 3   Deksripsi            1738 non-null   object
dtypes: int64(1), object(3)
memory usage: 60.0+ KB


In [4]:
fill_values = {'Nama Produk': '', 'Harga': '', 'Deskripsi':''}

df_preprocess.fillna('-', inplace=True)
df_preprocess


,Sub Kategori Produk,Nama Produk,Harga,Deksripsi
0,Permen,Supermen,12000,-
1,-,-,16000,<p>Permen susu khas lombok nusa tenggara barat...
2,Coklat,Semeton Choco (Coklat Lombok),15000,-
3,Kopi,Kopi Lanang,19000,<p>Kopi Lanang Bubuk Ground&nbsp;<br>Level Gri...
4,Kopi,Kopi Lombok Murni,28000,<p>Kopi denga kemasaran Pouch <br>100% Tanpa C...
...,...,...,...,...
1911,Madu,Mbojo Honey,169000,<p>Mbojo Honey<br></p>
1912,Kopi,Kopi Bubuk 100 Gr,29000,<p>Kopi Bubuk Pesanggerahan 100 Gr</p><p><span...
1913,Kopi,Kopi Jahe 100 Gr,29000,<p>Kopi Jahe Pesanggerahan 100 Gr</p><p><span ...
1914,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,<p>Dompet Motif Krealang Sumbawa<br></p>


In [25]:
df_preprocess['Sub Kategori Produk'] = df_preprocess['Sub Kategori Produk'].astype(str)
df_preprocess['Nama Produk'] = df_preprocess['Nama Produk'].astype(str)
df_preprocess['Harga'] = df_preprocess['Harga'].astype(str)
df_preprocess['Deksripsi'] = df_preprocess['Deksripsi'].astype(str)
df_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Sub Kategori Produk  1916 non-null   object
 1   Nama Produk          1916 non-null   object
 2   Harga                1916 non-null   object
 3   Deksripsi            1916 non-null   object
 4   Concatenated         1916 non-null   object
dtypes: object(5)
memory usage: 75.0+ KB


In [6]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    text_without_tags = soup.get_text()
    return ' '.join(text_without_tags.split())

# Apply the function to the DataFrame column
df_preprocess['Deksripsi'] = df_preprocess['Deksripsi'].apply(remove_html_tags)
df_preprocess

,Sub Kategori Produk,Nama Produk,Harga,Deksripsi
0,Permen,Supermen,12000,-
1,-,-,16000,Permen susu khas lombok nusa tenggara barat ya...
2,Coklat,Semeton Choco (Coklat Lombok),15000,-
3,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...
4,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...
...,...,...,...,...
1911,Madu,Mbojo Honey,169000,Mbojo Honey
1912,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...
1913,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...
1914,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa


In [7]:
df_preprocess['Concatenated'] = df_preprocess.apply(lambda row: ' '.join(row), axis=1)
df_preprocess

,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated
0,Permen,Supermen,12000,-,Permen Supermen 12000 -
1,-,-,16000,Permen susu khas lombok nusa tenggara barat ya...,- - 16000 Permen susu khas lombok nusa tenggar...
2,Coklat,Semeton Choco (Coklat Lombok),15000,-,Coklat Semeton Choco (Coklat Lombok) 15000 -
3,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...,Kopi Kopi Lanang 19000 Kopi Lanang Bubuk Groun...
4,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...,Kopi Kopi Lombok Murni 28000 Kopi denga kemasa...
...,...,...,...,...,...
1911,Madu,Mbojo Honey,169000,Mbojo Honey,Madu Mbojo Honey 169000 Mbojo Honey
1912,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...,Kopi Kopi Bubuk 100 Gr 29000 Kopi Bubuk Pesang...
1913,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...,Kopi Kopi Jahe 100 Gr 29000 Kopi Jahe Pesangge...
1914,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa,Fashion Wanita Dompet Motif Krealang Sumbawa 4...


In [8]:
for i in range(len(df_preprocess)):
  df_preprocess['Nama Produk'][i] = df_preprocess['Nama Produk'][i].lower()
  df_preprocess['Harga'][i] = df_preprocess['Harga'][i].lower()
  df_preprocess['Deksripsi'][i] = df_preprocess['Deksripsi'][i].lower()


In [9]:
def ganti_isi_column(data_lama,data_baru):
  for i in range(len(data_lama)) :
    data_lama[i] = data_baru[i]
  print(data_lama)

In [10]:
def preprocessing(document_to_be_scored):
  # Tokenize Dataset
  pisah = {}
  for i in range(len(document_to_be_scored)):
    pisah[i] = document_to_be_scored[i].split() #melakukan split

  # Menghilangkan stopword memakai library nltk
  stopword=['dengan'] # custom list stopword

  removed = {}
  hasil = []
  listStopword = stopwords.words('indonesian') # list stopword dari NLTK

  # Tambahkan stopword
  for i in stopword:
    listStopword.append(i)
  print(listStopword,"\n")

  # listStopword = set(stopwords.words('indonesian')) #list stopword
  hitung = 0 #untuk menghitung frekuensi
  for i in pisah.values():
    for j in i:
      if j not in listStopword: #jika setiap kata tidak ada di list stopword
        hasil.append(j)
    removed[hitung] = hasil[:]
    del hasil[:]
    hitung=hitung+1

  # Memakai library sastarawi
  # Memakai library sastrawi untuk melakukan stemming
  factory = StemmerFactory() #membuat objek stemmer
  stemmer = factory.create_stemmer()
  hitung = 0

  hasil=[]
  hasil2 = {}

  for i in removed.values():
    for j in i:
      # Hasil Stemming
      hasil.append(stemmer.stem(j))
    hasil2[hitung] = hasil[:]
    del hasil[:]
    hitung = hitung+1
  hasil =""
  korpus2 =[]
  for i in hasil2.values():
    hasil =""
    for j in i:
      hasil = hasil+ " " +j
    korpus2.append(hasil)
  #Timpa hasil
  ganti_isi_column(document_to_be_scored,korpus2)

8 minutes run

In [11]:
preprocessing(df_preprocess['Concatenated'])

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [12]:
df_preprocess
# df.to_csv('your_file.csv', index=False)

,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated
0,Permen,supermen,12000,-,permen supermen 12000 -
1,-,-,16000,permen susu khas lombok nusa tenggara barat ya...,- - 16000 permen susu khas lombok nusa tengga...
2,Coklat,semeton choco (coklat lombok),15000,-,coklat semeton choco coklat lombok 15000 -
3,Kopi,kopi lanang,19000,kopi lanang bubuk ground level grinder coffee ...,kopi kopi lanang 19000 kopi lanang bubuk grou...
4,Kopi,kopi lombok murni,28000,kopi denga kemasaran pouch 100% tanpa campuran...,kopi kopi lombok murni 28000 kopi denga kemas...
...,...,...,...,...,...
1911,Madu,mbojo honey,169000,mbojo honey,madu mbojo honey 169000 mbojo honey
1912,Kopi,kopi bubuk 100 gr,29000,kopi bubuk pesanggerahan 100 gr#kopi #kopilomb...,kopi kopi bubuk 100 gr 29000 kopi bubuk sangg...
1913,Kopi,kopi jahe 100 gr,29000,kopi jahe pesanggerahan 100 gr#kopi #kopilombo...,kopi kopi jahe 100 gr 29000 kopi jahe sangger...
1914,Fashion Wanita,dompet motif krealang sumbawa,455000,dompet motif krealang sumbawa,fashion wanita dompet motif krealang sumbawa ...


In [13]:
def compute_tfidf(documents):
    # Step 1: Tokenization
    def tokenize(document):
        return document.lower().split()

    # Step 2: Create a vocabulary
    vocabulary = set()
    for doc in documents:
        vocabulary.update(tokenize(doc))

    vocabulary = sorted(vocabulary)

    # Step 3: Compute TF (Term Frequency)
    print("\nIterasi pada setiap data:")
    count=1
    tf_matrix = []
    for doc in documents:
        tokens = tokenize(doc)
        tf = [tokens.count(word) for word in vocabulary]
        print("Data Ke-",count,":",tokens)
        tf_matrix.append(tf)
        count+=1


    # Step 4: Compute IDF (Inverse Document Frequency)
    idf = []
    N = len(documents)
    for word in vocabulary:
        doc_count = sum([1 for doc in documents if word in tokenize(doc)])
        idf.append(math.log(N / (1 + doc_count)))


    # Step 5: Compute TF-IDF
    tfidf_matrix = []
    for tf_vector in tf_matrix:
        tfidf = [tf * idf_value for tf, idf_value in zip(tf_vector, idf)]
        tfidf_matrix.append(tfidf)
    
    return vocabulary, tf_matrix, idf, tfidf_matrix

In [14]:
documents = df_preprocess['Concatenated']

vocabulary,tf_matrix,idf,tfidf_matrix = compute_tfidf(documents)


Iterasi pada setiap data:
Data Ke- 1 : ['permen', 'supermen', '12000', '-']
Data Ke- 2 : ['-', '-', '16000', 'permen', 'susu', 'khas', 'lombok', 'nusa', 'tenggara', 'barat', 'sehat', 'kandung', 'gula', 'rusak', 'citra', 'susu', 'permen']
Data Ke- 3 : ['coklat', 'semeton', 'choco', 'coklat', 'lombok', '15000', '-']
Data Ke- 4 : ['kopi', 'kopi', 'lanang', '19000', 'kopi', 'lanang', 'bubuk', 'ground', 'level', 'grinder', 'coffee', 'fine', 'halus', 'roasting', 'medium', 'to', 'darknetto', '100', 'g']
Data Ke- 5 : ['kopi', 'kopi', 'lombok', 'murni', '28000', 'kopi', 'denga', 'kemasaran', 'pouch', '100', 'tanpa', 'campur', 'netto', '170', 'gr']
Data Ke- 6 : ['fashion', 'wanita', 'lestary', 'tenun', 'batik', 'sasambo', 'blouse', 'kuning', '610000', 'lestary', 'tenun', 'batik', 'sasambo', 'nlouse', 'kuning', 'size', 'm']
Data Ke- 7 : ['kaos', 'kaos', 'mandalika', 'sirkuit', '150000', 'kaos', 'mandalika', 'sirkuit']
Data Ke- 8 : ['kaos', 'kaos', 'mandalika', '137500', 'kaos', 'mandalika']
Data

In [15]:
import math
import csv

csv_file_path = 'tfidf_features.csv'

# Save the TF-IDF matrix to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row (vocabulary)
    writer.writerow(vocabulary)

    # Write each TF-IDF vector as a row
    for tfidf_vector in tfidf_matrix:
        writer.writerow(tfidf_vector)

print(f'TF-IDF features saved to {csv_file_path}')

TF-IDF features saved to tfidf_features.csv


In [16]:
def precision(positive,negative,label_document_1):
    print("Nilai positif prediksi  = ",positive)
    print("Nilai negative prediksi = ",negative)

    if(label_document_1 == 'positive'):
        precision = positive/limit
        print('Precision = {}/{} = {}'.format(positive,limit,precision))
    else:
        precision = negative/limit
        print('Precision = {}/{} = {}'.format(negative,limit,precision))
    return precision

In [17]:
df_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Sub Kategori Produk  1916 non-null   object
 1   Nama Produk          1916 non-null   object
 2   Harga                1916 non-null   object
 3   Deksripsi            1916 non-null   object
 4   Concatenated         1916 non-null   object
dtypes: object(5)
memory usage: 75.0+ KB


In [18]:
query =''
query = input('masukkan query: ')

print(query)

camilan khas lombok


In [19]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory() #membuat objek stemmer
stemmer = factory.create_stemmer()
query =stemmer.stem(query)

In [20]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Mengembalikan nilai cosine similarity dari dua teks yang diberikan
def compute_cosine_similarity(text1, text2):

    # list
    list_text = [text1, text2]

    # converts text into vectors with the TF-IDF
    vectorizer = TfidfVectorizer()
    vectorizer.fit_transform(list_text)
    tfidf_text1, tfidf_text2 = vectorizer.transform([list_text[0]]), vectorizer.transform([list_text[1]])

    # computes the cosine similarity
    cs_score = cosine_similarity(tfidf_text1, tfidf_text2)

    return np.round(cs_score[0][0],5)

In [23]:
similarity = {}
i =0

for doc in df_preprocess['Concatenated']: #similarities antara input dan review
  similarity[i]= compute_cosine_similarity(query,doc)
  i = i+1
output = sorted(similarity.items(), key=lambda x:x[1], reverse=True)
print(output)
print("menampilkan 5 Data paling relevan (nilai cosine similiarity tertinggi) dengan "+ "\'" +query+"\'" + "\n\n")
i =0
count = 0
limit = 5
positive =0
negative =0
for k, v in output:
  #jika nilai scoring dibawah 0.01 skip
  if v < 0.01 :continue 
  if(i >= limit): break
  try:
    similarity = "{:.5f}".format(v)
    print("Pada record ke ",str(k))
    print("Nilai similaritas : ", similarity)
    print("Nama Produk       : ", df_preprocess['Nama Produk'][k])
    print("Deskripsi         : ", df_preprocess['Deksripsi'][k])
    print("\n")

    # if(df_tfidf['label'][k] == 'positif'):
    #     positive+=1
    # else:
    #     negative+=1    

  except:
    continue
  i=i+1

print("\033[1m" + "Cosine Similarity" + "\033[0m")
print("Label Dokumen Uji = ",label_document_1)
# precision(positive,negative,label_document_1)

[(127, 0.34677), (1821, 0.32704), (78, 0.27967), (1352, 0.25039), (1800, 0.24323), (1381, 0.24224), (403, 0.23774), (1559, 0.23314), (108, 0.22842), (1808, 0.2227), (1358, 0.21863), (639, 0.21701), (1802, 0.21693), (1806, 0.21693), (126, 0.21507), (228, 0.20592), (354, 0.20592), (455, 0.20403), (1801, 0.20199), (1803, 0.20199), (1813, 0.20199), (1807, 0.19766), (1815, 0.19766), (1758, 0.18443), (355, 0.1842), (356, 0.1842), (1354, 0.1842), (29, 0.18275), (31, 0.18275), (1831, 0.18275), (1353, 0.1806), (90, 0.17304), (319, 0.17078), (1, 0.16817), (72, 0.16368), (77, 0.16368), (75, 0.15976), (517, 0.15976), (519, 0.15976), (524, 0.15976), (526, 0.15976), (529, 0.15976), (81, 0.15954), (1819, 0.15866), (747, 0.15853), (750, 0.15853), (1351, 0.15662), (1472, 0.15368), (1488, 0.14771), (684, 0.14686), (1738, 0.14663), (1739, 0.14663), (702, 0.14567), (582, 0.14497), (1104, 0.14489), (679, 0.14366), (98, 0.14292), (352, 0.14292), (1554, 0.14292), (1384, 0.13994), (1355, 0.13993), (1697, 0.13

NameError: name 'label_document_1' is not defined

In [36]:
index_document = 23

# Baca file CSV yang berisi fitur TF-IDF
df = pd.read_csv('tfidf_features.csv')

# Pilih kolom yang berisi vektor TF-IDF, misalnya 'tfidf_vector'
tfidf_matrix = tfidf_matrix = df.iloc[::] 

# Fungsi untuk menghitung Euclidean Distance antara dua vektor
def euclidean_distance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

# Hitung kemiripan dokumen 1 dengan dokumen lainnya
document_1 = tfidf_matrix.iloc[index_document]  # Ganti dengan indeks dokumen yang ingin Anda bandingkan
label_document_1 = df_preprocess['Sub Kategori Produk'][index_document]

print(df_preprocess["Nama Produk"][index_document])
print(label_document_1)

#digunakan untuk menyimpan hasil perhitungan jarak Euclidean antara dokumen referensi dan dokumen-dokumen lainnya.
similarities = []

for i in range(1, len(tfidf_matrix)):
    document_i = tfidf_matrix.iloc[i]
    distance = euclidean_distance(document_1, document_i)
    similarities.append((i, distance))

# Sort hasil kemiripan berdasarkan jarak (semakin kecil semakin mirip)
similarities.sort(key=lambda x: x[1])

# Tampilkan dokumen yang paling mirip dengan dokumen pertama
count = 0
limit = 200
positive =0
negative =0
for i, distance in similarities:
    print(f"Dokumen {i}: \nEuclidean Distance = {distance}")
    print("Label           : ",df_preprocess['Sub Kategori Produk'][i])
    print("Review          :",df_preprocess['Concatenated'][i])

    print("\n")
    if (count==limit): break
    else: count+=1

    if(df_preprocess['Sub Kategori Produk'][i] == 'positif'):
        positive+=1
    else:
        negative+=1    

print("\033[1m" + "Euclidean Distance" + "\033[0m")
print("Label Dokumen Uji = ",label_document_1)
print("Precision = {}".format(precision(positive,negative,label_document_1)))

tas tenun tbs taa00
Fashion Wanita
Dokumen 23: 
Euclidean Distance = 0.0
Label           :  Fashion Wanita
Review          :  fashion wanita tas tenun tbs taa00 1800000 tas tenun tbs taa00


Dokumen 22: 
Euclidean Distance = 13.72969555594172
Label           :  Fashion Wanita
Review          :  fashion wanita tas tenun tbs 1800000 tas tenun tbs


Dokumen 1887: 
Euclidean Distance = 21.661503102713585
Label           :  Fashion Wanita
Review          :  fashion wanita jaket tenun 1500000 -


Dokumen 11: 
Euclidean Distance = 21.714369574529247
Label           :  Tas
Review          :  tas tas tenun 437500 tas songket ungu tenun lombok timur


Dokumen 311: 
Euclidean Distance = 21.732774494266106
Label           :  Fashion Wanita
Review          :  fashion wanita tas ketak tenun 500000 tas ketak tenun


Dokumen 233: 
Euclidean Distance = 21.920296443130994
Label           :  Kerajinan Bambu
Review          :  rajin bambu tas bambu kombinasi tenun 180000 tas bambu kombinasi tenun


Dokume